**DESAFIO ENACOM**

**OBJETIVO**: Prever a geração de energia mensal!

In [1]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Obtendo os dados no DataBase

Os dados foram obtidos a partir do site de Dados Abertos ONS, disponível em: https://dados.ons.org.br/dataset/geracao-usina-2

In [2]:
# Criar listas de anos e números
anos = list(range(2000, 2021)) # Para o desafio, deve-se considerar o range até 2021, para a previsão do próximo mês no período atual, deve-se inserir 2025
numeros = list(range(1, 13))

# Criar função para obter os dados de um ano específico, considerando apenas ano
def download_csv_ano(ano):
    try:
      if ano <= 2021:
        url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_{ano}.csv"
        # Imprimir a URL antes de fazer o download
        print(f"URL: {url}")
        return pd.read_csv(url, sep=';')
    except Exception as e:
        print(f"Erro ao baixar dados de {url}: {e}")
        return None

# Criar função para obter os dados de um ano específico, considerando ano e número do mês
def download_csv_ano_numero(ano, numero):
    try:
      if ano > 2021:
        if numero < 10:
            url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA-2_{ano}_0{numero}.csv"
        else:
            url = f"https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA-2_{ano}_{numero}.csv"
        # Imprimir a URL antes de fazer o download
        print(f"URL: {url}")
        return pd.read_csv(url, sep=';')
    except Exception as e:
        print(f"Erro ao baixar dados de {url}: {e}")
        return None

# Baixar os DataFrames com apenas no ano (correspodem os dados de 2000 a 2021)
dado_df_ano = pd.concat([download_csv_ano(ano) for ano in anos], ignore_index=True)

# OBS: Os próximos scripts a seguir podem exigir um consumo maior de memória RAM.
# Recomenda-se a alocação de uma quantidade maior de memória na máquina para executá-los com sucesso.
# Vale ressaltar que a versão gratuita do Google Colab pode não oferecer suporte suficiente para executar
# esses scripts devido ao consumo aumentado de memória.

# Baixar os DataFrames com o ano e o número. (correspodem os dados de 2022 e 2023)
#dado_df_ano_numero = pd.concat([download_csv_ano_numero(ano, num) for ano in anos for num in numeros], ignore_index=True) # Para rodar, remova-se "#"

# Concatenar os dois DataFrames (correspodem os dados desde de 2000 a 2023)
#dado_df = pd.concat([dado_df_ano, dado_df_ano_numero], ignore_index=True) # Para rodar, remova-se "#"

URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2000.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2001.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2002.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2003.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2004.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2005.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2006.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2007.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GERACAO_USINA_2008.csv
URL: https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/geracao_usina_2_ho/GER

In [43]:
# Visualizando o dataframe
dado_df_ano.tail()

,din_instante,id_subsistema,id_estado,nom_tipousina,val_geracao
59203582,2021-12-31 23:00:00,SE,SP,TÉRMICA,0.000
59203583,2021-12-31 23:00:00,SE,SP,TÉRMICA,147.000
59203584,2021-12-31 23:00:00,SE,TO,HIDROELÉTRICA,233.109
59203585,2021-12-31 23:00:00,SE,TO,HIDROELÉTRICA,584.262
59203586,2021-12-31 23:00:00,SE,TO,HIDROELÉTRICA,420.073


# Análise dos dados

In [3]:
# Eliminar colunas desnecessárias
dado_df_ano.drop(columns=['nom_subsistema','nom_estado','cod_modalidadeoperacao','nom_tipocombustivel','nom_usina','ceg','id_ons'], inplace=True)

In [4]:
# Verificar oS tipos de dados em cada coluna
dado_df_ano.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59203587 entries, 0 to 59203586
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   din_instante   object 
 1   id_subsistema  object 
 2   id_estado      object 
 3   nom_tipousina  object 
 4   val_geracao    float64
dtypes: float64(1), object(4)
memory usage: 2.2+ GB


In [5]:
# Verificar se há valores ausentes
dado_df_ano.isna().sum()

din_instante          0
id_subsistema         0
id_estado             0
nom_tipousina         0
val_geracao      171633
dtype: int64

In [6]:
# Eliminar os NaNs
dado_df_ano.dropna(inplace=True)

In [7]:
# Converter a coluna 'din_instante' para datetime
dado_df_ano['din_instante'] = pd.to_datetime(dado_df_ano['din_instante'])

In [8]:
# Agregar os dados por mês, ano, região, estado e tipo de usina
dado_df = dado_df_ano.assign(
    mes_ano = dado_df_ano['din_instante'].dt.to_period('M'),  # Cria uma nova coluna 'mes_ano'
).groupby(['mes_ano', 'id_subsistema', 'id_estado', 'nom_tipousina']).agg( # Agrupa por mes_ano, região, estado e tipo de geração
    {'val_geracao': 'sum'}  # soma a geração de energia
).reset_index()  # Reinicia o índice do DataFrame resultante

In [ ]:
# Imprimir o resumo da estatística descritiva (VISÃO GERAL)
print('O resumo da estatística descritiva (VISÃO GERAL):')
print(dado_df['val_geracao'].describe())

O resumo da estatística descritiva (VISÃO GERAL):
count    1.228800e+04
mean     8.415035e+05
std      1.315219e+06
min      0.000000e+00
25%      4.560380e+04
50%      2.875353e+05
75%      9.702424e+05
max      1.105588e+07
Name: val_geracao, dtype: float64


In [9]:
# Segregar o datagrame por região (POR REGIÃO)
dado_regiao = dado_df[['mes_ano','id_subsistema','nom_tipousina','val_geracao']]

In [10]:
# Imprimir o resumo da estatística descritiva (POR REGIÃO)

# Imprimindo as estatísticas descritivas de cada região
print("Resumo da região Norte (N):")
print(dado_regiao[dado_regiao['id_subsistema']=='N']['val_geracao'].describe())
print('')

print("Resumo da região Nordeste (NE):")
print(dado_regiao[dado_regiao['id_subsistema']=='NE']['val_geracao'].describe())
print('')

print("Resumo da região Sul (S):")
print(dado_regiao[dado_regiao['id_subsistema']=='S']['val_geracao'].describe())
print('')

print("Resumo da região Sudeste (SE):")
print(dado_regiao[dado_regiao['id_subsistema']=='SE']['val_geracao'].describe())

Resumo da região Norte (N):
count    1.076000e+03
mean     1.004757e+06
std      1.523876e+06
min      0.000000e+00
25%      6.608689e+04
50%      3.555285e+05
75%      1.399251e+06
max      1.105588e+07
Name: val_geracao, dtype: float64

Resumo da região Nordeste (NE):
count    3.485000e+03
mean     4.028340e+05
std      5.360076e+05
min      0.000000e+00
25%      2.146033e+04
50%      1.441738e+05
75%      6.128324e+05
max      2.794819e+06
Name: val_geracao, dtype: float64

Resumo da região Sul (S):
count    1.888000e+03
mean     8.466869e+05
std      1.030282e+06
min      0.000000e+00
25%      1.854425e+05
50%      4.041947e+05
75%      1.159424e+06
max      4.593161e+06
Name: val_geracao, dtype: float64

Resumo da região Sudeste (SE):
count    5.839000e+03
mean     1.071563e+06
std      1.589260e+06
min      0.000000e+00
25%      4.486695e+04
50%      3.204046e+05
75%      1.274677e+06
max      7.560476e+06
Name: val_geracao, dtype: float64


In [11]:
# Calcular a matriz de correlação entre as regiões
matriz_correlacao = dado_regiao.pivot_table(index='mes_ano', columns='id_subsistema', values='val_geracao', aggfunc='sum').corr()

# Exibir a matriz de correlação
print("Matriz de Correlação entre as Regiões:")
print(matriz_correlacao)

Matriz de Correlação entre as Regiões:
id_subsistema         N        NE         S        SE
id_subsistema                                        
N              1.000000  0.462095  0.105691  0.506662
NE             0.462095  1.000000  0.302723  0.448338
S              0.105691  0.302723  1.000000  0.327463
SE             0.506662  0.448338  0.327463  1.000000


In [12]:
# Calcular a matriz de correlação entre as fontes de geração
matriz_correlacao = dado_regiao.pivot_table(index='mes_ano', columns='nom_tipousina', values='val_geracao', aggfunc='sum').corr()

# Exibir a matriz de correlação
print("Matriz de Correlação entre as Fontes de geração:")
print(matriz_correlacao)

Matriz de Correlação entre as Fontes de geração:
nom_tipousina  BOMBEAMENTO  EOLIELÉTRICA  FOTOVOLTAICA  HIDROELÉTRICA  \
nom_tipousina                                                           
BOMBEAMENTO       1.000000      0.267035     -0.112744       0.008532   
EOLIELÉTRICA      0.267035      1.000000      0.782256      -0.395071   
FOTOVOLTAICA     -0.112744      0.782256      1.000000      -0.150824   
HIDROELÉTRICA     0.008532     -0.395071     -0.150824       1.000000   
NUCLEAR          -0.020724      0.121918     -0.039489       0.153214   
TÉRMICA          -0.280717      0.628543      0.251974       0.066048   

nom_tipousina   NUCLEAR   TÉRMICA  
nom_tipousina                      
BOMBEAMENTO   -0.020724 -0.280717  
EOLIELÉTRICA   0.121918  0.628543  
FOTOVOLTAICA  -0.039489  0.251974  
HIDROELÉTRICA  0.153214  0.066048  
NUCLEAR        1.000000  0.305408  
TÉRMICA        0.305408  1.000000  


In [13]:
# Visualizar graficamente os dados (VISÃO GERAL)

# Agrupar por região para facilitar a visualização do gráfico
df_plot = dado_regiao.groupby(['mes_ano', 'id_subsistema'])['val_geracao'].sum().reset_index()

# Converter a columa mes_ano em strings
df_plot['mes_ano'] = df_plot['mes_ano'].astype(str)

# Plotar o gráfico
fig = px.bar(df_plot, x='mes_ano', y='val_geracao', color='id_subsistema',
             title='Evolução da Geração de Energia por Região (Visão Geral)',
             labels={'val_geracao': 'Geração de Energia (MW med)', 'mes_ano': 'Data'},
             color_discrete_map={'N': 'blue', 'NE': 'black', 'S': 'orange', 'SE': 'purple'})

fig.update_xaxes(title_text='Data')
fig.update_yaxes(title_text='Geração de Energia (MW med)')
fig.update_layout(legend_title_text='Regiões')
fig.show()

In [14]:
# Visualizar graficamente os dados (VISÃO GERAL)

# Agrupar por região para facilitar a visualização do gráfico
plot_fonte = dado_regiao.groupby(['mes_ano', 'nom_tipousina'])['val_geracao'].sum().reset_index()

# Converter a columa mes_ano em strings
plot_fonte['mes_ano'] = plot_fonte['mes_ano'].astype(str)

# Plotar o gráfico
fig = px.bar(plot_fonte, x='mes_ano', y='val_geracao', color='nom_tipousina',
             title='Evolução da Geração de Energia por Fonte de Geração (Visão Geral)',
             labels={'val_geracao': 'Geração de Energia (MW med)', 'mes_ano': 'Data'},
             color_discrete_map={'EOLIELÉTRICA': 'blue', 'NUCLEAR': 'black', 'TÉRMICA': 'orange', 'HIDROELÉTRICA': 'purple','BOMBEAMENTO':'red', 'FOTOVOLTAICA':'brown'})

fig.update_xaxes(title_text='Data')
fig.update_yaxes(title_text='Geração de Energia (MW med)')
fig.update_layout(legend_title_text='Regiões')
fig.show()

In [15]:
# Por Região (Nordeste)

# Agrupar os dados por tipo de geração e
plot_por_regiao = dado_regiao.groupby(['mes_ano', 'nom_tipousina','id_subsistema'])['val_geracao'].sum().reset_index()

# Convertendo a coluna 'mes_ano' para string
plot_por_regiao['mes_ano'] = plot_por_regiao['mes_ano'].astype(str)

# Criar função plotar gráfico
def plotar_grafico(nome, sigla):
    fig = px.bar(plot_por_regiao[plot_por_regiao['id_subsistema']== f'{sigla}'], x='mes_ano', y='val_geracao', color='nom_tipousina',
                 title=f'Evolução da Geração de Energia na Região {nome.upper()}',
                 labels={'val_geracao': 'Geração de Energia (MW med)', 'mes_ano': 'Data'},
                 color_discrete_map={'EOLIELÉTRICA': 'blue', 'NUCLEAR': 'black', 'TÉRMICA': 'orange', 'HIDROELÉTRICA': 'purple','BOMBEAMENTO':'red', 'FOTOVOLTAICA':'brown'})

    # Calculando a média da região
    media_regiao = plot_por_regiao[plot_por_regiao['id_subsistema'] == f'{sigla}']['val_geracao'].mean()
    media_movel = plot_por_regiao[plot_por_regiao['id_subsistema'] == f'{sigla}']['val_geracao'].rolling(window=12).mean()

    # Adicionando a média como uma linha horizontal
    fig.add_trace(go.Scatter(x=plot_por_regiao[plot_por_regiao['id_subsistema'] == f'{sigla}']['mes_ano'], y=media_movel, mode='lines', name='Média Móvel', line=dict(color='green')))
    fig.add_hline(y=media_regiao, line_color="red", annotation_text=f'Média: {media_regiao:.2f} MW med', annotation_position="bottom right")

    fig.update_xaxes(title_text='Data')
    fig.update_yaxes(title_text='Geração de Energia (MW med)')
    fig.update_layout(legend_title_text='Formas de geração')
    fig.show()

In [16]:
# Plotando o gráfico da região Nordeste
plotar_grafico('nordeste', 'NE')

In [17]:
# Plotando o gráfico da região Norte
plotar_grafico('norte', 'N')

In [18]:
# Plotando o gráfico da região Sudeste
plotar_grafico('sudeste', 'SE')

In [19]:
# Plotando o gráfico da região Sul
plotar_grafico('sul', 'S')

# Modelos Preditivos

## Regressão Linear Múltipla

In [20]:
# Importando as bibliotecas necessárias
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Criando variáveis independentes (X) e dependente (y)
X = dado_df[['id_subsistema', 'nom_tipousina']]
y = dado_df['val_geracao']

# Converter colunas categóricas em variáveis dummy
X_dummy = pd.get_dummies(X, columns=['id_subsistema', 'nom_tipousina'])

# Dividir os dados em conjunto de treinamento e teste após a codificação
X_train, X_test, y_train, y_test = train_test_split(X_dummy, y, test_size=0.2, random_state=42)

# Treinar o modelo de regressão linear
regressao_linear = LinearRegression()
regressao_linear.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_linear = regressao_linear.predict(X_test)

# Avaliar o modelo
lin_mse = mean_squared_error(y_test, y_pred_linear)
print("Mean Squared Error:", np.sqrt(lin_mse))

Mean Squared Error: 1156045.1058225331


## Random Forest

In [22]:
# Importando a biblioteca necessária
from sklearn.ensemble import RandomForestRegressor

# Treinar o modelo Random Forest
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_forest = random_forest.predict(X_test)

# Avaliar o modelo
forest_mse = mean_squared_error(y_test, y_pred_forest)
print("Mean Squared Error:", np.sqrt(forest_mse))

Mean Squared Error: 1140206.7560821392


## Decision Tree

In [23]:
# Importando a biblioteca necessária
from sklearn.tree import DecisionTreeRegressor

# Criar e treinar o modelo MLPRegressor
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_decision = decision_tree.predict(X_test)

# Avaliar o modelo
tree_mse = mean_squared_error(y_test, y_pred_decision)
print("Mean Squared Error:", np.sqrt(tree_mse))

Mean Squared Error: 1140076.0798514606


**Comentário**: O modelo de Árvore de Decisão apresentou melhor métrica do Erro Quadrático Médio em relação os demais modelos, é importante realizar a validação cruzada (cross-validation) para avaliar o desempenho do modelo de forma mais robusta. A validação cruzada permite estimar a capacidade de generalização do modelo para dados não vistos, fornecendo uma avaliação mais confiável de sua performance.

## Cross- Validation

In [28]:
# Importando a biblioteca necessária
from sklearn.model_selection import cross_val_score

# Regressão Linear

# Calculando os scores de validação cruzada usando a Regressão Linear
scores = cross_val_score(regressao_linear, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
lin_rmse_scores = np.sqrt(-scores)

# Definindo uma função para exibir os scores
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

# Exibindo os scores de validação cruzada
display_scores(lin_rmse_scores)

Scores: [1142728.66301122 1191954.34657602 1133046.21187582 1199881.03968996
 1128986.04757395 1047312.2240007  1145906.08025944 1195663.81913087
 1152905.82872455 1098190.02439662]
Mean: 1143657.4285239144
Standard deviation: 44684.82288937716


In [34]:
# Random Forest

# Calculando os scores de validação cruzada usando a Floresta Aleatória
scores = cross_val_score(random_forest, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
forest_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(forest_rmse_scores)

Scores: [1132590.57430805 1173699.32728382 1116461.48520248 1180989.3426077
 1115536.10739554 1035819.26057724 1129973.35937742 1177296.07359523
 1139753.83148344 1087246.02303049]
Mean: 1128936.5384861417
Standard deviation: 42340.774849939364


In [33]:
# Decision Tree

# Calculando os scores de validação cruzada usando a árvore de decisão
scores = cross_val_score(decision_tree, X_train, y_train,
                         scoring="neg_mean_squared_error", cv=10)

# Calculando a raiz quadrada negativa do erro quadrático médio para cada fold
tree_rmse_scores = np.sqrt(-scores)

# Exibindo os scores de validação cruzada
display_scores(tree_rmse_scores)

Scores: [1132609.00208345 1173690.10005516 1116471.90397115 1181022.28992202
 1115502.57769447 1035865.18462978 1130023.49158223 1177282.96723986
 1139720.39047431 1087332.56883589]
Mean: 1128952.0476488306
Standard deviation: 42323.92595060022


**Resultado**: O modelo Random Forest obteve o melhor desempenho, pois apresentou a menor média do Erro Quadrático Médio.

## Otimização de hiperparâmetros

In [32]:
# Importando a biblioteca necessária
from sklearn.model_selection import GridSearchCV

# Definir a grade
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

# Criar um estimador
forest_reg = RandomForestRegressor()

# Criar um objeto GridSearchCV
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# Ajustar o modelo usando GridSearchCV
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [35]:
# Exibindo os melhores parâmetros encontrados no Grid Search
grid_search.best_params_

{'max_features': 4, 'n_estimators': 10}

In [36]:
# Retorna o melhor estimador encontrado no Grid Search
grid_search.best_estimator_

RandomForestRegressor(max_features=4, n_estimators=10)

In [38]:
# Atribui o melhor modelo encontrado pelo Grid Search
final_model = grid_search.best_estimator_

# Realiza previsões no conjunto de teste usando o melhor modelo encontrado
final_model_predictions = final_model.predict(X_test)

# Calcula o Mean Squared Error (MSE) entre as previsões e os valores reais do conjunto de teste
final_mse = mean_squared_error(y_test, final_model_predictions)

# Imprime a raiz quadrada do MSE
print(np.sqrt(final_mse))

1139950.7636921997


In [41]:
# Cria uma figura contendo dois gráficos de linhas: um para os valores reais e outro para as previsões do modelo final
fig = go.Figure(data=[go.Scatter(y=y_test.values, name='Real'),
                      go.Scatter(y=final_model_predictions, name='Previsões')])

# Atualiza os nomes dos eixos x e y
fig.update_xaxes(title_text='Amostras')
fig.update_yaxes(title_text='Valor')

fig.show()

In [56]:
# Definindo o próximo mês para previsão
proximo_mes = '2022-01'

# Codificando o próximo mês
proximo_mes_encoded = pd.get_dummies(pd.Series([proximo_mes]), prefix='', prefix_sep='')

# Reindexando as colunas do próximo mês para corresponder às colunas do conjunto de treinamento
proximo_mes_encoded = proximo_mes_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Fazer previsões para o próximo mês usando o modelo final
previsao_proximo_mes = final_model.predict(proximo_mes_encoded)

# Exibir a previsão
print("Previsão para o próximo mês: {:.2f} MW med".format(previsao_proximo_mes[0]))

Previsão para o próximo mês: 157010.30 MW med
